In [1]:
from flask import Flask, render_template, jsonify
from kafka import KafkaConsumer
import json
import pandas as pd
import plotly.express as px

app = Flask(__name__)

# Initialize Kafka consumer
consumer = KafkaConsumer('housing_data', bootstrap_servers=['localhost:9092'])

def process_data():
    # Fetch data from Kafka topic and process it
    data = []
    for message in consumer:
        data.append(json.loads(message.value.decode('utf-8')))
        if len(data) >= 10:
            break

    # Convert to DataFrame for visualization purposes
    df = pd.DataFrame(data)

    # Prepare chart data for bar chart (total dwellings over time)
    years = df['year']
    total_dwelling = df['total_dwelling']

    # Prepare data for line chart (construction starts over time)
    line_data = None
    if 'total_constn_start' in df.columns:
        line_data = df[['year', 'total_constn_start', 'region']]

    # Prepare data for the table
    table_data = df[[
        'year', 'total_dwelling', 'single_detached', 'multiple', 'semi_detached',
        'row', 'apartment', 'total_dwelling_market', 'homeownership_freehold', 'rental'
    ]]

    # Format 'year' column to display as floats with decimal points
    table_data['year'] = table_data['year'].astype(str)  # Convert year to string
    table_data['year'] = table_data['year'].str.replace('.0', '', regex=False)  # Remove decimal point

    return {
        'years': years.tolist(),
        'total_dwelling': total_dwelling.tolist(),
        'line_data': line_data.to_dict(orient='records') if line_data is not None else None,
        'table_data': table_data.to_dict(orient='records')
    }


@app.route('/')
def index():
    chart_data = process_data()
    return render_template('index.html', chart_data=chart_data)

In [2]:
if __name__ == '__main__':
    try:
        app.run(debug=False)
    except Exception as e:
        print("An error occurred:", str(e))

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\jahir\AppData\Local\Temp\ipykernel_31044\2372869498.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_data['year'] = table_data['year'].astype(str)  # Convert year to string
C:\Users\jahir\AppData\Local\Temp\ipykernel_31044\2372869498.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_data['year'] = table_data['year'].str.replace('.0', '', regex=False)  # Remove decimal point
127.0.0.1 - - [11/Apr/2024 16:27:14] "GET / HTTP/1